In [ ]:
# intallation
%pip install --quiet google-ai-generativelanguage==0.6.15
%pip install --upgrade --quiet google-generativeai
%pip install --upgrade --quiet langchain_community
%pip install --upgrade --quiet PyPDF2
%pip install --upgrade --quiet langchain-chroma
%pip install --upgrade --quiet pypdf

In [1]:
%pip install --upgrade --quiet langchain_community

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# imports
import getpass
import os

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains.llm import LLMChain
# LangChain I/O
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# LCEL
from langchain.schema.runnable import RunnablePassthrough
# Summerize document
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
import textwrap

hello


In [ ]:
# Load documents
file_path = './Corporate Governance in India.pdf'
loader = PyPDFLoader(file_path)
documents = loader.load()
print(f"Loaded {len(documents)} documents from the folder.")

In [ ]:
# setup Google Generative AI
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API key here ")

In [ ]:
# create LLM instance
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [ ]:
# test
test_llm_response = llm.invoke("Tell me a joke about economics")
print(test_llm_response.content)

In [ ]:
# create embeddings model
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [ ]:
# Create Chroma vector store from documents
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="my_document_collection",
    persist_directory="/content/chroma_db"
)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever.invoke("What this corporate gevernance?")

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt
)
response = rag_chain.invoke("What this corporate gevernance?")

In [ ]:
def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
question = "What this corporate gevernance?"
response = rag_chain.invoke(question)
print(response)

In [ ]:
question = "How to achieve corporate gevernance in corporations?"
response = rag_chain.invoke(question)
print(response)

In [ ]:
# Summerize document
prompt_template2 = """Write a concise summary and with important concepts of the following:
"{text}"
"""
prompt2 = PromptTemplate.from_template(prompt_template2)

In [ ]:
llm_chain2 = LLMChain(llm=llm, prompt=prompt2)

In [ ]:
stuff_chain2 = StuffDocumentsChain(
    llm_chain=llm_chain2,
    document_variable_name="text"
  )

In [ ]:
output_summary2 = stuff_chain2.run(documents)

In [ ]:
wrapped_text2 = textwrap.fill(output_summary2,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text2)